### Quora Question Pairs
是一个二分类问题，旨在判断一对问题是否具有相同的含义。这项任务的目标是识别出在意义上相似或者相同的问题，即判断它们是否可以被视为同一个问题的不同表述。
#### 任务特点
1. 输入：每个样本包含两个问题（例如，"What is the best way to learn Python?" 和 "How can I effectively learn Python?"）。
2. 输出：一个二进制标签，指示这对问题是否重复或相似（1 表示相似，0 表示不相似）。
3. 应用场景：该任务在问答系统、搜索引擎优化和用户生成内容的去重中具有广泛应用。

### Transformers完成Quora Question Pairs任务的步骤
1. 数据准备
* 输入格式：将每对问题组合成输入，通常格式为 "[CLS] question1 [SEP] question2 [SEP]"。[CLS] 是分类标记，[SEP] 用于分隔两个问题。
* 标签：为每对问题分配标签（1 表示相似，0 表示不相似）。
2. Tokenization
* 使用预训练的 tokenizer（如 BERT 的 tokenizer）将文本转换为模型可以理解的输入格式。这包括：
    * 将文本转换为词 ID。
    * 创建注意力掩码（attention mask），指示模型哪些部分是实际输入，哪些部分是填充（padding）。
3. 嵌入层
* 输入的词 ID 通过嵌入层（embedding layer）转换为密集向量，这些向量包含了词的语义信息。
4. Transformer编码器
* 输入向量通过多个 Transformer 编码器层，这些层包括自注意力机制（self-attention）和前馈神经网络（feedforward neural network）。
    * 自注意力机制：允许模型在处理一个问题时，同时考虑输入中所有词的关系，从而捕捉上下文信息。
    * 层归一化和残差连接：提高模型的稳定性和收敛速度。
5. 池化
* 在最后一层输出中，使用 [CLS] 标记的输出向量作为整个问题对的表示。这通常是进行分类的基础。
6. 分类层
* 将 [CLS] 的输出通过一个全连接层（fully connected layer）进行分类，输出一个二分类结果（相似或不相似）。
7. 训练
* 使用交叉熵损失（cross-entropy loss）来评估模型的输出与实际标签之间的差距。通过反向传播（backpropagation）更新模型参数，逐步提高准确性。
8. 评估与推理
* 在验证集上评估模型性能，使用指标如准确率（accuracy）、F1 分数等来衡量模型效果。

### Tokenization的底层实现
1. 文本预处理
* 在开始分词之前，通常需要对文本进行预处理。这包括去除多余的空格、转换为小写（对于某些模型）、去除特殊字符等。
2. 分词算法
* WordPiece（用于 BERT）：将单词分解为更小的子词单元，允许模型处理未见过的词。
* Byte Pair Encoding (BPE)（用于 GPT）：通过合并频率最高的字符对来创建词汇表，能够适应多种语言。
* SentencePiece：一种无监督的分词算法，可以处理空格不明显的语言，如中文。
3. 构建词汇表
* 在训练过程中，Tokenization 通常会构建一个词汇表，记录每个 token 及其对应的 ID。这一步骤在使用预训练模型时通常已经完成。
4. 分词与编码
* 将文本输入 Tokenizer，使用分词算法将文本拆分为 token。
* 为每个 token 分配一个 ID，生成 input_ids。
* 例如，在 WordPiece 中，"unhappiness" 可能被拆分为 ["un", "happiness"]，并分别分配 ID。
5. 生成Attention Mask
* 对于填充的 token，生成一个 attention mask，指示哪些 tokens 是有效的（1）和无效的（0）。这对于处理可变长度的输入很重要。
6. 支持特殊token
* Tokenizer 通常支持添加一些特殊 token，比如 [CLS]（分类 token）和 [SEP]（分隔 token），这在多句子输入时尤为重要。

### Embedding Layer的底层实现
主要涉及将离散的词或标识符（ID）映射到连续的向量空间。这一过程通常包括以下几个步骤：
1. 词汇表建立
* 在训练之前，首先要构建一个词汇表，记录每个词或 token 及其对应的唯一 ID。这可以是从训练数据中统计得出的。
* Embedding Layer的词汇表通常是从Tokenization过程中生成的词汇表。
2. 初始化嵌入矩阵
* 嵌入层通常用一个二维矩阵表示，行数为词汇表大小，列数为嵌入维度。每一行对应一个词的嵌入向量。
* 这个嵌入矩阵通常是随机初始化的，或者使用预训练的嵌入（如 Word2Vec、GloVe）。
3. 查找嵌入向量
* 当输入一个序列的 token ID 时，嵌入层会使用这些 ID 在嵌入矩阵中查找对应的行，得到相应的嵌入向量。
* 例如，对于输入的 ID 序列 [1, 2, 3]，嵌入层将返回对应的嵌入向量矩阵，其中每一行是 ID 对应的嵌入。
4. 前向传播
* 在神经网络中，嵌入层的输出将作为输入传递到后续的网络层。这个过程是模型的前向传播的一部分。
5. 反向传播
* 在训练过程中，嵌入层的权重（即嵌入矩阵）会随着损失的反向传播而更新。通过梯度下降等优化算法，嵌入向量会逐渐学习到更适合下游任务的表示。


### Transformers和Bert的关系
1. Transformers
* 定义：Transformers 是一种深度学习模型架构，首次在论文《Attention is All You Need》中提出。它们主要用于处理序列数据，特别是在自然语言处理（NLP）任务中。
* 组成部分：Transformers 由编码器（encoder）和解码器（decoder）组成，但在许多应用中（如文本分类和问答），通常只使用编码器部分。
* 自注意力机制：Transformers 的核心特性是自注意力机制（self-attention），使模型能够在处理输入序列时考虑到序列中所有单词的关系，从而捕捉上下文信息。
2. Bert
* 定义：BERT（Bidirectional Encoder Representations from Transformers）是基于 Transformer 编码器架构的预训练模型，专门为 NLP 任务设计。
* 双向性：BERT 的一个重要特点是其双向性，它在训练时考虑上下文中所有词的关系，能够同时使用左侧和右侧的上下文信息。
* 预训练和微调：BERT 先进行大规模文本的无监督预训练，然后通过微调（fine-tuning）来适应特定任务（如文本分类、问答等）。<br>

总结：BERT 是一种基于 Transformers 架构的特定模型，利用自注意力机制和双向上下文理解能力来处理各种自然语言处理任务。可以认为 BERT 是 Transformers 的一个具体实现，专注于增强语言表示的能力。

### Bert在Quora Question Pairs任务中希望的输入
1. 输入对
* 两个问题（question1 和 question2）需要作为输入
2. 输入格式
* input_ids：对应两个问题的 token ID，使用 BERT 的 tokenizer 进行编码。
* attention_mask：指示哪些 tokens 是实际输入（1）和填充（0）。
* token_type_ids：用于区分两个句子的标识，question1 的 token_type_ids 为 0，question2 的 token_type_ids 为 1。
3. 标签
* 一个标签（labels），指示这两个问题是否重复（0 或 1）。

输入示例：<br>
{<br>
    "input_ids": [101, 2054, 2003, ...],  // Token IDs<br>
    "attention_mask": [1, 1, ...],       // Attention mask<br>
    "token_type_ids": [0, 0, ..., 1, 1, ...],  // Token type IDs<br>
    "labels": 0  // 0: 不重复, 1: 重复<br>
}<br>


### 1. 导包

In [1]:
import pandas as pd
from datasets import load_dataset
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import torch

/Users/bytedance/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/bytedance/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 2. 加载数据集

In [20]:
# 加载数据集
dataset = load_dataset('quora', split='train', trust_remote_code=True)

In [5]:
dataset

Dataset({
    features: ['questions', 'is_duplicate'],
    num_rows: 404290
})

In [6]:
dataset_df=pd.DataFrame(dataset)

In [7]:
dataset_df.head()

,questions,is_duplicate
0,"{'id': [1, 2], 'text': ['What is the step by s...",False
1,"{'id': [3, 4], 'text': ['What is the story of ...",False
2,"{'id': [5, 6], 'text': ['How can I increase th...",False
3,"{'id': [7, 8], 'text': ['Why am I mentally ver...",False
4,"{'id': [9, 10], 'text': ['Which one dissolve i...",False


In [10]:
dataset_df['questions'][0]

{'id': [1, 2],
 'text': ['What is the step by step guide to invest in share market in india?',
  'What is the step by step guide to invest in share market?']}

### 3. 数据预处理

In [21]:
# 提取问题对和标签
def extract_questions_and_labels(examples):
    questions = examples['questions']
    return {
        'question1': questions['text'][0],  # 获取第一个问题文本
        'question2': questions['text'][1],  # 获取第二个问题文本
        'is_duplicate': examples['is_duplicate']  # 复制标签
    }

In [22]:
# 使用 map 函数转换数据集
dataset = dataset.map(extract_questions_and_labels)

In [23]:
dataset

Dataset({
    features: ['questions', 'is_duplicate', 'question1', 'question2'],
    num_rows: 404290
})

In [24]:
dataset_df=pd.DataFrame(dataset)

In [25]:
dataset_df.head()

,questions,is_duplicate,question1,question2
0,"{'id': [1, 2], 'text': ['What is the step by s...",False,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...
1,"{'id': [3, 4], 'text': ['What is the story of ...",False,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...
2,"{'id': [5, 6], 'text': ['How can I increase th...",False,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...
3,"{'id': [7, 8], 'text': ['Why am I mentally ver...",False,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...
4,"{'id': [9, 10], 'text': ['Which one dissolve i...",False,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?


In [14]:
# 初始化 tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [36]:
# 数据预处理，tokenization 和添加 labels
def preprocess_function(examples):
    # 对问题对进行编码
    encoding = tokenizer(
        examples['question1'],
        examples['question2'],
        truncation=True,
        padding='max_length',
        max_length=128
    )
    # 返回编码结果和标签
    return {
        'input_ids': encoding['input_ids'],
        'attention_mask': encoding['attention_mask'],
        'labels': [1 if label else 0 for label in examples['is_duplicate']] # 添加 labels
    }


In [37]:
# 应用预处理
tokenized_dataset = dataset.map(preprocess_function, batched=True)



Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.



Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.




Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. 

In [28]:
tokenized_dataset

Dataset({
    features: ['questions', 'is_duplicate', 'question1', 'question2', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 404290
})

In [38]:
# 剔除questions和is_duplicate
# 剔除不需要的字段
tokenized_dataset = tokenized_dataset.remove_columns(['questions', 'is_duplicate'])

# 检查最终的数据集
print(tokenized_dataset.column_names)  # 确认保留的字段


['question1', 'question2', 'input_ids', 'attention_mask', 'labels']


In [39]:
tokenized_dataset

Dataset({
    features: ['question1', 'question2', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 404290
})

In [40]:
print(tokenized_dataset[0])  # 打印第一个样本以确认字段


{'question1': 'What is the step by step guide to invest in share market in india?', 'question2': 'What is the step by step guide to invest in share market?', 'input_ids': [101, 2054, 2003, 1996, 3357, 2011, 3357, 5009, 2000, 15697, 1999, 3745, 3006, 1999, 2634, 1029, 102, 2054, 2003, 1996, 3357, 2011, 3357, 5009, 2000, 15697, 1999, 3745, 3006, 1029, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

#### 最后得到的tokenized_dataset
1. input_ids
* 描述：每个问题对被编码为模型输入的ID列表
* 示例：[101, 2054, 2003, 1996, ...]（数字对应于 BERT 词汇表中的词）。
2. attention_mask
* 描述：指示哪些 tokens 是实际输入（1）以及哪些是填充（0）。
* 示例：[1, 1, 1, 1, 0, 0, ...]（有效 token 用 1 表示，填充用 0 表示）。
3. labels
* 描述：二分类标签，表示两个问题是否重复（0 或 1）。
* 示例：0（表示这两个问题不重复）或 1（表示它们重复）。
4. 其他可选字典
* 如果你在数据预处理过程中保留了问题文本，可以包含 question1 和 question2 字段用于进一步分析。

### 4. 设置模型和训练参数

In [41]:
# 设置模型
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# 设置训练参数
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    logging_dir='./logs',
    logging_steps=10,
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### 5. 模型训练

跑的时间比较久所以先不跑了

In [42]:
# 使用 Trainer 进行训练
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_dataset  # 可以替换为验证集
)

# 开始训练
trainer.train()

  0%|          | 10/75807 [00:05<10:25:30,  2.02it/s]

{'loss': 0.664, 'grad_norm': 4.907032489776611, 'learning_rate': 4.9993404303032704e-05, 'epoch': 0.0}


  0%|          | 20/75807 [00:10<10:08:38,  2.08it/s]

{'loss': 0.717, 'grad_norm': 2.8049395084381104, 'learning_rate': 4.9986808606065405e-05, 'epoch': 0.0}


  0%|          | 30/75807 [00:14<10:08:53,  2.07it/s]

...

{'loss': 0.5383, 'grad_norm': 12.814064979553223, 'learning_rate': 4.9195324969989584e-05, 'epoch': 0.05}


  2%|▏         | 1230/75807 [09:49<9:50:57,  2.10it/s]

{'loss': 0.3363, 'grad_norm': 6.220242500305176, 'learning_rate': 4.9188729273022285e-05, 'epoch': 0.05}


  2%|▏         | 1236/75807 [09:52<9:44:59,  2.12it/s]

KeyboardInterrupt: 